In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

In [ ]:
RUNS = { "noghc": "decr_14_3h_ws",
        "annual": "decr_16_3h_ws",
        "monthly": "decr_15_3h_ws",
        "hourly": "decr_13_3h_ws",
       }

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
#OUTPUT = PREFIX + "results/" + RUN + "/graphics/comp_benefits/"

In [ ]:
h2export = 100
opts = "Co2L2.0"
ghrules = ["noghc","annual","monthly", "hourly"]

In [ ]:
# Filter stats according to h2export value and opts value
def get_data(stats, export, opts):
    cost = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"cost"]
    mg_AC_exclu_H2 = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_AC_exclu_H2 El_all"]
    mg_H2_False_False_exportonly = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_H2_False_False_exportonly"]
    return cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly 

In [ ]:
df = pd.DataFrame(index=ghrules, columns=["cost", "mg_AC_exclu_H2", "mg_H2_False_False_exportonly"])

for rule in ghrules:
    INPUT = PREFIX + "results/" + RUNS[rule] + "/integrated_comp/stats.csv"
    stats = pd.read_csv(INPUT, index_col=0)
    cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly = get_data(stats, h2export, opts)    
    df.loc[rule, "cost"] = cost.values[0]
    df.loc[rule, "mg_AC_exclu_H2"] = mg_AC_exclu_H2.values[0]
    df.loc[rule, "mg_H2_False_False_exportonly"] = mg_H2_False_False_exportonly.values[0]    

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

#ax.plot(df.index, df["cost"]/1e9, label="Total cost")
ax.plot(df.index, df["mg_AC_exclu_H2"], label="local electricity price")
ax.plot(df.index, df["mg_H2_False_False_exportonly"], label="h2 costs paid by exporters")

# Add secondary axis for total cost
ax2 = ax.twinx()
ax2.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")
ax2.set_ylabel("Total cost in B€")
ax2.set_ylim(10, 26)

ax.set_xlabel("H2 rule")
ax.set_ylabel("Marginal price of electricity/h2 in €/MWh")
ax.set_ylim(0, 100)
# ax.set_xlim(0, 90)
#ax.grid(axis='y')

ax.legend()
ax2.legend()

In [ ]:
#ax.legend()
if not os.path.isdir(OUTPUT):
    os.makedirs(OUTPUT)
    print("Created directory " + OUTPUT)
plt.savefig(OUTPUT + "electricity" + str(export) + ".pdf", bbox_inches="tight")
plt.show()